# Data Collection and Pre processing
The aim of this notebook is to collect data from various sources and process it so that we can analyse or predict in the later stages. 

**Objective of the study:** Analyze and predict Energy demand for facilities at Arizona State University(ASU), Tempe Campus.

In [1]:
import pandas as pd
from pathlib import Path
import datetime
import numpy as np
import glob
import pickle

## Starting with one building file
Starting with data file of one of the buildings from 69 facilities at ASU to explore, analyse and to scale up the code for all the buildings.

'Armstrong' is the name of the building with gross floor area of 105,232 Sq.ft.

Armstrong.xls, along with other xls files for all the facilities are exported from the Energy management system of ASU. This file contains energy demand data in kW on hourly interval basis for an entire year starting from Jan 01, 2019 until Dec 31, 2019.

Here, xls[0] gives the first sheet of the excel file

In [2]:
xls = pd.read_html('Data/EnergyDataFiles/armstrong.xls')
xls[0]

,0,1,2
0,EIS Charting Data Dump: Timeframe = Last Year ...,EIS Charting Data Dump: Timeframe = Last Year ...,EIS Charting Data Dump: Timeframe = Last Year ...
1,NaN,Armstrong - kW Avg,NaN
2,Tue Jan 1 00:00:00 GMT-0700 2019,61.44999980926514,0
3,Tue Jan 1 01:00:00 GMT-0700 2019,60.70832443237305,0
4,Tue Jan 1 02:00:00 GMT-0700 2019,64.32917594909668,0
...,...,...,...
8757,Tue Dec 31 19:00:00 GMT-0700 2019,55.76457500457764,0
8758,Tue Dec 31 20:00:00 GMT-0700 2019,54.13544940948486,0
8759,Tue Dec 31 21:00:00 GMT-0700 2019,53.32290077209473,0
8760,Tue Dec 31 22:00:00 GMT-0700 2019,52.19582557678223,0


### Extracting the name of the building from the xls file
Locating the name of the building and extracting it using string methods - split() and strip()

In [3]:
name = xls[0].loc[1,1].split('-')[0].strip()
name

'Armstrong'

### Extracting the Energy demand values from the file
Extracting the column with label as '1' and dropping the first two rows using .drop(index = []). Then, the Series object is converted to a Dataframe with Column name as 'Energy'

In [4]:
energy = xls[0][1].drop(index = [0,1]).reset_index(drop = True)
energy = energy.to_frame(name = 'Energy')
energy

,Energy
0,61.44999980926514
1,60.70832443237305
2,64.32917594909668
3,61.05000019073486
4,62.6583251953125
...,...
8755,55.76457500457764
8756,54.13544940948486
8757,53.32290077209473
8758,52.19582557678223


### Loading Buildings database
This buildings database contains the details of each building at ASU. The useful variables for our study includes:
1. Gross Flooe Area (GSF)
2. Building Name (To map details with the energy file)
3. Building Type (Type)

Building type includes types like:
    a. Academic
    b. Support
    c. Housing
    d. Library etc

In [5]:
Buildings = pd.read_csv('Data/BuildingDatabase.csv')
Buildings

,Building Name,Building \nNumber,Status,Type,Campus,Abbreviation,Address,GSF
0,GAMMAGE AUDITORIUM,1,atleast one project,Academic,Tempe,GGMA,1200 S. FOREST AVENUE,150097.0
1,INTERDISCIPLINARY A,2,atleast one project,Academic,Tempe,INTDSA,1100 S. CADY MALL,28332.0
2,INTERDISCIPLINARY B,3,atleast one project,Academic,Tempe,INTDSB,1120 S. CADY MALL,62941.0
3,SHESC,4,atleast one project,Academic,Tempe,SHESC,900 S. CADY MALL,49078.0
4,SOCIAL SCIENCES BLDG.,5,No projects so far,Academic,Tempe,SS,951 S. CADY MALL,87673.0
...,...,...,...,...,...,...,...,...
456,Physical Education Building West,PHW,atleast one project,NaN,Polytechnic,PEBW,NaN,NaN
457,ALAMEDA BUILDING,ALA,atleast one project,NaN,Tempe,ALA,734 West Alameda Dr,NaN
458,CENTERPOINT,CNT,atleast one project,Academic,Tempe,CNT,660 S. Mill Ave.,NaN
459,COMMUNITY SERVICES,CSB,atleast one project,Services,Tempe,CSB,"200 E Curry Rd, Tempe, AZ 85281",109322.0


### Extracting the building gross floor area and building type using the buidling name from Energy data file
Using building name as primary key, Building's Gross Floor Area in Sq.ft and BuildingType is extracted.

In [6]:
gsf = Buildings.GSF[Buildings['Building Name'].str.contains(f'{name}',regex = False, case = False)].reset_index(drop = True)
Btype = Buildings.Type[Buildings['Building Name'].str.contains(f'{name}',regex = False, case = False)].reset_index(drop = True)
print(f'Building GSF:{gsf[0]}\nBuilding type:{Btype[0]}')

Building GSF:105232.0
Building type:Academic


Copying and filling the Name, GSF, Btype values of the builing to join it with Energy data Dataframe

In [7]:
df = pd.DataFrame()
df['Name'] = np.full(energy.shape[0], name)
df['GFA'] = np.full(energy.shape[0], gsf[0])
df['BType'] = np.full(energy.shape[0], Btype[0])
df

,Name,GFA,BType
0,Armstrong,105232.0,Academic
1,Armstrong,105232.0,Academic
2,Armstrong,105232.0,Academic
3,Armstrong,105232.0,Academic
4,Armstrong,105232.0,Academic
...,...,...,...
8755,Armstrong,105232.0,Academic
8756,Armstrong,105232.0,Academic
8757,Armstrong,105232.0,Academic
8758,Armstrong,105232.0,Academic


# Gathering all other data
Information like outside tempearure and Semester schedule is required in order to predict the energy demand more accurately. Semester schedule along with weekday and Gross floor area provide a rough estimate for Building Occupancy. 

> Temperature data is obtained from NOAA database and Semester information is obtained from Arizona State University's academic calendar

### Loading temperature Data
The weather data contains an elaborate number of metrics which are captured at local Pheonix weather station.
Temperature metrics TAVG (Average Temperature), TMAX and TMIN are important for our study.

In [8]:
TempFile = pd.ExcelFile('Data/CompleteTemperatureProfile.xlsx')
TempDF = TempFile.parse(0)
TempDF

,STATION,NAME,DATE,AWND,PGTM,PRCP,SNOW,SNWD,TAVG,TMAX,...,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT03,WT07,WT08,WT10
0,USW00023183,"PHOENIX AIRPORT, AZ US",2018-01-01,3.36,NaN,0.0,0.0,0.0,56.0,73,...,100.0,340.0,10.1,13.0,NaN,NaN,NaN,NaN,NaN,NaN
1,USW00023183,"PHOENIX AIRPORT, AZ US",2018-01-02,4.47,NaN,0.0,0.0,0.0,60.0,75,...,50.0,40.0,10.1,13.0,NaN,NaN,NaN,NaN,NaN,NaN
2,USW00023183,"PHOENIX AIRPORT, AZ US",2018-01-03,5.14,NaN,0.0,0.0,0.0,61.0,76,...,70.0,60.0,17.0,23.0,NaN,NaN,NaN,NaN,NaN,NaN
3,USW00023183,"PHOENIX AIRPORT, AZ US",2018-01-04,3.58,NaN,0.0,0.0,0.0,66.0,79,...,90.0,60.0,12.1,19.0,NaN,NaN,NaN,NaN,NaN,NaN
4,USW00023183,"PHOENIX AIRPORT, AZ US",2018-01-05,4.25,NaN,0.0,0.0,0.0,63.0,77,...,130.0,130.0,10.1,12.1,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
921,USW00023183,"PHOENIX AIRPORT, AZ US",2020-07-10,11.41,NaN,0.0,0.0,0.0,101.0,111,...,280.0,280.0,23.0,28.0,NaN,NaN,NaN,NaN,NaN,NaN
922,USW00023183,"PHOENIX AIRPORT, AZ US",2020-07-11,10.07,NaN,0.0,0.0,0.0,103.0,115,...,240.0,280.0,21.0,29.1,NaN,NaN,NaN,NaN,NaN,NaN
923,USW00023183,"PHOENIX AIRPORT, AZ US",2020-07-12,7.16,NaN,0.0,0.0,NaN,104.0,116,...,180.0,160.0,17.0,23.9,NaN,NaN,NaN,NaN,NaN,NaN
924,USW00023183,"PHOENIX AIRPORT, AZ US",2020-07-13,7.61,NaN,0.0,0.0,0.0,105.0,114,...,270.0,240.0,17.0,23.0,NaN,NaN,NaN,NaN,NaN,NaN


In [9]:
TempDF.describe()

,AWND,PGTM,PRCP,SNOW,SNWD,TAVG,TMAX,TMIN,WDF2,WDF5,WSF2,WSF5,WT01,WT02,WT03,WT07,WT08,WT10
count,925.000000,34.000000,926.000000,867.0,817.0,920.000000,926.000000,926.000000,925.000000,924.000000,925.000000,924.000000,51.0,1.0,52.0,14.0,39.0,1.0
mean,6.517838,1525.176471,0.020270,0.0,0.0,75.375000,87.049676,63.739741,209.924324,220.898268,17.232973,22.418723,1.0,1.0,1.0,1.0,1.0,1.0
std,2.322185,510.836715,0.131509,0.0,0.0,15.850829,16.341217,15.198374,89.866993,96.442431,6.054289,7.677499,0.0,NaN,0.0,0.0,0.0,NaN
min,2.010000,2.000000,0.000000,0.0,0.0,41.000000,47.000000,30.000000,10.000000,10.000000,6.900000,8.900000,1.0,1.0,1.0,1.0,1.0,1.0
25%,4.700000,1437.750000,0.000000,0.0,0.0,61.750000,73.000000,51.000000,120.000000,150.000000,13.000000,17.000000,1.0,1.0,1.0,1.0,1.0,1.0
50%,6.260000,1644.000000,0.000000,0.0,0.0,74.000000,86.000000,62.000000,260.000000,260.000000,16.100000,21.000000,1.0,1.0,1.0,1.0,1.0,1.0
75%,7.830000,1738.750000,0.000000,0.0,0.0,91.000000,103.000000,77.000000,280.000000,290.000000,21.000000,25.900000,1.0,1.0,1.0,1.0,1.0,1.0
max,19.010000,2328.000000,2.360000,0.0,0.0,105.000000,116.000000,94.000000,360.000000,360.000000,52.100000,70.900000,1.0,1.0,1.0,1.0,1.0,1.0


#### Extracting 2019 year's temperature profile

In [10]:
TempDF['Year'] = TempDF['DATE'].apply(lambda x: x.year)

In [11]:
Temp = TempDF.loc[TempDF['Year'] == 2019,['DATE','TAVG','TMIN','TMAX']].reset_index(drop=True)
Temp['DATE'] = Temp['DATE'].astype(str)
Temp

,DATE,TAVG,TMIN,TMAX
0,2019-01-01,43.0,38,51
1,2019-01-02,41.0,30,52
2,2019-01-03,43.0,33,57
3,2019-01-04,48.0,37,67
4,2019-01-05,54.0,45,67
...,...,...,...,...
360,2019-12-27,49.0,46,51
361,2019-12-28,49.0,41,55
362,2019-12-29,44.0,35,53
363,2019-12-30,47.0,39,56


### Loading Semester Schedule

In [12]:
Sem = pd.read_csv('Data/Semester_Schedule.csv')
Sem.rename(columns = {'Semester/Holiday': 'Sem'},inplace = True)
Sem

,Date,Sem,Details
0,1/1/19,0,Break
1,1/2/19,0,Break
2,1/3/19,0,Break
3,1/4/19,0,Break
4,1/5/19,0,Break
...,...,...,...
360,12/27/19,0,Break
361,12/28/19,0,Break
362,12/29/19,0,Break
363,12/30/19,0,Break


### Gathering Temperature and Semester daily values together

In [13]:
DailyDF = Temp.join(Sem.drop(columns = ['Date','Details']))
DailyDF

,DATE,TAVG,TMIN,TMAX,Sem
0,2019-01-01,43.0,38,51,0
1,2019-01-02,41.0,30,52,0
2,2019-01-03,43.0,33,57,0
3,2019-01-04,48.0,37,67,0
4,2019-01-05,54.0,45,67,0
...,...,...,...,...,...
360,2019-12-27,49.0,46,51,0
361,2019-12-28,49.0,41,55,0
362,2019-12-29,44.0,35,53,0
363,2019-12-30,47.0,39,56,0


### Creating the Hourly Interval Dataframe along with weekday

The Temperature profile and Semester Schedule data is available on a daily basis. Therefore, this hourly interval dataframe is used to assign daily averages to hourly intervals. datetime's weekday method is used to generate weekdays.

In [14]:
HourInterval = pd.date_range('2019-01-01',periods = 8760,freq='H').to_frame(name = 'DateTime').reset_index(drop =True)
HourInterval['WeekDay'] = HourInterval['DateTime'].apply(lambda x:x.weekday())
HourInterval

,DateTime,WeekDay
0,2019-01-01 00:00:00,1
1,2019-01-01 01:00:00,1
2,2019-01-01 02:00:00,1
3,2019-01-01 03:00:00,1
4,2019-01-01 04:00:00,1
...,...,...
8755,2019-12-31 19:00:00,1
8756,2019-12-31 20:00:00,1
8757,2019-12-31 21:00:00,1
8758,2019-12-31 22:00:00,1


As we want the information about weekdays, let's map the weekdays to 1 and weekends to 0

In [15]:
HourInterval['WeekDay'] = HourInterval['WeekDay'].map({0:1,1:1,2:1,3:1,4:1,5:0,6:0})
HourInterval

,DateTime,WeekDay
0,2019-01-01 00:00:00,1
1,2019-01-01 01:00:00,1
2,2019-01-01 02:00:00,1
3,2019-01-01 03:00:00,1
4,2019-01-01 04:00:00,1
...,...,...
8755,2019-12-31 19:00:00,1
8756,2019-12-31 20:00:00,1
8757,2019-12-31 21:00:00,1
8758,2019-12-31 22:00:00,1


#### Attaching Daily values dataframe to hourly interval and  Weekday values

In [16]:
HourInterval['Date'] = HourInterval['DateTime'].astype(str).apply(lambda x: x.split(' ')[0])
HourInterval['Hour'] = HourInterval['DateTime'].astype(str).apply(lambda x: x.split(' ')[1])

In [17]:
infoDF = HourInterval.merge(DailyDF,left_on = 'Date',right_on = 'DATE')

In [18]:
infoDF.drop(columns = ['DateTime','Date','DATE'],inplace = True)
infoDF

,WeekDay,Hour,TAVG,TMIN,TMAX,Sem
0,1,00:00:00,43.0,38,51,0
1,1,01:00:00,43.0,38,51,0
2,1,02:00:00,43.0,38,51,0
3,1,03:00:00,43.0,38,51,0
4,1,04:00:00,43.0,38,51,0
...,...,...,...,...,...,...
8755,1,19:00:00,54.0,45,59,0
8756,1,20:00:00,54.0,45,59,0
8757,1,21:00:00,54.0,45,59,0
8758,1,22:00:00,54.0,45,59,0


#### Checking by joining the energy dataframe with other information dataframe

In [19]:
df = df.join([energy,infoDF])
df

,Name,GFA,BType,Energy,WeekDay,Hour,TAVG,TMIN,TMAX,Sem
0,Armstrong,105232.0,Academic,61.44999980926514,1,00:00:00,43.0,38,51,0
1,Armstrong,105232.0,Academic,60.70832443237305,1,01:00:00,43.0,38,51,0
2,Armstrong,105232.0,Academic,64.32917594909668,1,02:00:00,43.0,38,51,0
3,Armstrong,105232.0,Academic,61.05000019073486,1,03:00:00,43.0,38,51,0
4,Armstrong,105232.0,Academic,62.6583251953125,1,04:00:00,43.0,38,51,0
...,...,...,...,...,...,...,...,...,...,...
8755,Armstrong,105232.0,Academic,55.76457500457764,1,19:00:00,54.0,45,59,0
8756,Armstrong,105232.0,Academic,54.13544940948486,1,20:00:00,54.0,45,59,0
8757,Armstrong,105232.0,Academic,53.32290077209473,1,21:00:00,54.0,45,59,0
8758,Armstrong,105232.0,Academic,52.19582557678223,1,22:00:00,54.0,45,59,0


# Scaling the code for all facilities

In [20]:
def extractxls (x, Buildings, dailyDF):
    xls = pd.read_html(x)
    name = xls[0].loc[1,1].split('-')[0].strip()
    print(name)
    energy = xls[0][1].drop(index = [0,1]).reset_index(drop = True)
    energy = energy.to_frame(name = 'Energy')
    gsf = Buildings.GSF[Buildings['Building Name'].str.contains(f'{name}',regex = False, case = False)].reset_index(drop = True)
    Btype = Buildings.Type[Buildings['Building Name'].str.contains(f'{name}',regex = False, case = False)].reset_index(drop = True)
    df = pd.DataFrame()
    df['Name'] = np.full(energy.shape[0], name)
    df['GFA'] = np.full(energy.shape[0], gsf[0])
    df['BType'] = np.full(energy.shape[0], Btype[0])
    return df.join([energy,infoDF])

In [21]:
Masterdf = pd.DataFrame()
for xlsheet in glob.glob('Data/EnergyDataFiles/*.xls'):
    Masterdf = Masterdf.append(extractxls(xlsheet,Buildings,infoDF)).reset_index(drop =True)

Piper Writing Center
Cowden Family Resources
Hayden Hall
GLV Community Center
Gammage Aud
Family Studies
Social Sciences
Moeur
McClintock Hall
Fulton Parking
Schwada COB
Student Pavilion
Music
Dixie Gammage Hall
Lifescience A_B_D
PV West
Engineering Research Ctr
MU
Murdock Hall
Matthews Center
Matthews Hall
Old Main
University Center A Wing
Bus Admin C
Stauffer Hall
Bookstore
Bulldog Hall
PV East
Psychology North
Hayden Lib
Phys Sci G
Stauffer B Wing
McCord Hall
Phys Sci F
Phys Sci D
Education Lecture Hall
Armstrong
Phys Sci E
Discovery Hall
Phys Sci H
Sun Devil Sports Performance
Health Services
ASUPD
Interdisciplinary AB
Fulton Center
Carson Student Athletic Ctr
College Avenue Commons
Bio Design Institute C
Manzanita
Farrington Softball Stadium
Lang and Lit
Neeb Hall
Sun Devil Hall
Bus Admin AB
Payne Hall
Sun Devil Stadium
COD South
Mona Plummer Aquatics Ctr
Stauffer A Wing
Irish Hall
Lattie Coor Hall
SHESC
COD North
Engineering G
Farmer Education
Computing Commons
Lyceum Theater
Gol

## Saving the dataframe object to a pickle file for further exploration and prediction

In [22]:
with open("Data/DF.pkl", "wb") as file:
    pickle.dump(Masterdf, file)
    file.close()